In [1]:
%pip install llama-index-llms-huggingface
!pip install llama-index
!pip install transformers accelerate bitsandbytes
!pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("dataPrueba").load_data()

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig( # configuración de cuantificación del modelo -> reduce tamaño y aumenta velocidad
    load_in_4bit=True,                    # carga el modelo en formato 4bits
    bnb_4bit_compute_dtype=torch.float16, # más configuraciones relacionadas con eso
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(                                                             # se carga mistral desde hugging face
    model_name="mistralai/Mistral-7B-Instruct-v0.1",                              # modelo
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",                          # tokenizador
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),  # plantilla para envolver las consultas al modelo, lo que ayuda a guiar al modelo para responder de una manera específica
    context_window=3900,                                                          # tamaño máximo de la ventana de contexto que el modelo puede considerar para cada predicción
    max_new_tokens=256,                                                           # tokens que el modelo puede generar
    model_kwargs={"quantization_config": quantization_config},                    # configuración de cuantificación
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},              # parámetros para la generación de texto
    device_map="auto",                                                            # el modelo se distribuye automaticamente entre los dispositivos disponibles
    system_prompt="Always respond in Spanish."                                    # prompt para que conteste en español
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

<ipython-input-13-83586933102f>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


In [14]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [15]:
from llama_index.core.response.notebook_utils import display_response

In [16]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("¿Cuál es la misión de las fuerzas armadas?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The mission of the armed forces is to ensure the sovereignty and independence of Spain, defend its territorial integrity, and uphold the constitutional order.

In [18]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Cuál es la misión de las fuerzas armadas?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La misión de las Fuerzas Armadas es garantizar la soberanía e independencia de España, defender su integridad territorial y el ordenamiento constitucional. Además, las fuerzas armadas tienen la responsabilidad de defender la patria y sus intereses nacionales, tanto en el ámbito nacional como internacional. Ellas también tienen la misión de garantizar la seguridad y la protección de la población civil y de los recursos naturales del país.

In [19]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("¿Cuál es la capital de España?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The capital of Spain is Madrid.

In [20]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Cuál es la capital de España?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La capital de España es la villa de Madrid. Además, según el artículo 61 de la Constitución española, el Rey, al ser proclamado ante las Cortes Generales, prestará juramento de desempeñar fielmente sus funciones, guardar y hacer guardar la Constitución y las leyes y respetar los derechos de los ciudadanos y de las Comunidades Autónomas. El Rey también tiene la facultad de sancionar y promulgar las leyes, convocar y disolver las Cortes Generales y convocar elecciones en los términos previstos en la Constitución, entre otras funciones.